# Runge Kutta 4

In [126]:
import numpy as np
import matplotlib.pyplot as plt

G = 39.02 # Updated to work with Astronomical Units (AU)

SMU = lambda kg: kg  / 1.99e30 # Convert kg to Solar mass units (SMU)
YR = lambda seconds: seconds / 3.15e7 # Convert seconds to years (YR)
AU = lambda s: s * 6.67e-12 # Convert m to AU
AU_YR = lambda v: v * 2.1e-4 # Convert m/s to AU / YR

class Planet:
    def __init__(self, M : float, r : np.array, v : np.array):
        self.M = M
        self.vecSet = np.array([r, v])

class PlanetsContainer:
    def __init__(self, planets):
        r = [
            planet.vecSet[0] for planet in planets
        ]
        
        v = [
            planet.vecSet[1] for planet in planets
        ]
        
        M = [
            planet.M for planet in planets
        ]
        
        self.rTensor = np.array(r)
        self.vTensor = np.array(v)
        self.Tensor = np.array([r, v], float)
        self.M = np.array(M).T
        
        self.dim = self.Tensor.shape[-1]
        self.numBodies = len(planets)
    
    def updateTensors(self):
        # Fixes and updates the r and v tensors to match that of the components of the Tensor
        self.rTensor = self.Tensor[0]
        self.vTensor = self.Tensor[1]
        
        return None
    
    def accelerationTensor(self):
        # Builds the acceleration tensor corresponding to the changes dv on each body
        a = [[np.zeros(self.dim) for j in range(self.numBodies)] for i in range(self.numBodies)]
        
        # Fills the tensor with the corresponding components
        for i in range(self.numBodies):
            for j in range(self.numBodies):
                if i == j:
                    pass
                else:
                    a[i][j] = G * self.M[i] * self.M[j] / (np.linalg.norm(self.rTensor[j] - self.rTensor[i]) ** 2) \
                        * (self.rTensor[j] - self.rTensor[i]) / np.linalg.norm(self.rTensor[j] - self.rTensor[i])
        
        return np.array(a), np.sum(np.array(a), axis = 1)
    
    def f(self, r, dt):
        a = self.accelerationTensor()
        v = r[1] + a * dt
        return np.array([a, v], float)
    
    def RungeKutta4(self, tInitial, tFinal, h):
        t = tInitial
        T = []
        rS = [[], []]
        rE = [[], []]
        rM = [[], []]
        
        while t < tFinal:
            k1 = self.f(self.Tensor, h)
            k2 = self.f(self.Tensor + h / 2 * k1, h / 2)
            k3 = self.f(self.Tensor + h / 2 * k2, h / 2)
            k4 = self.f(self.Tensor + h * k3, t + h)
            
            self.Tensor += h / 6 * (k1 + 2 * (k2 + k3) + k4)
            print(h / 6 * (k1 + 2 * (k2 + k3) + k4))
            self.updateTensors()
            
            rS[0].append(self.Tensor[0][0][0])
            rS[1].append(self.Tensor[0][0][1])
            
            rE[0].append(self.Tensor[0][1][0])
            rE[1].append(self.Tensor[0][1][1])
            
            rM[0].append(self.Tensor[0][2][0])
            rM[1].append(self.Tensor[0][2][1])
            
            t += h
            T.append(t)
        
        return rE, T

# Sun
rS = np.array([0.0, 0.0])
vS = np.array([0.0, 0.0])
MS = 1

Sun = Planet(MS, rS, vS)

# Earth
rE = np.array([AU(1.474e11), 0.0])
vE = np.array([0.0, AU_YR(3e4)])
ME = SMU(5.972e24)

Earth = Planet(ME, rE, vE)

# Moon
rM = np.array([AU(3.844e8), 0.0]) + rE
vM = np.array([0.0, AU_YR(1.022e3)])
MM = SMU(3.748e22)

Moon = Planet(MM, rM, vM)

p = PlanetsContainer([Sun, Earth, Moon])
#r, t = p.RungeKutta4(0.0, 0.01, 1e-5)

#plt.plot(t, r[1])
#plt.show()

p.accelerationTensor()

(array([[[ 0.00000000e+00,  0.00000000e+00],
         [ 1.21145518e-04,  0.00000000e+00],
         [ 7.56353668e-07,  0.00000000e+00]],
 
        [[-1.21145518e-04,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00],
         [ 3.35491922e-07,  0.00000000e+00]],
 
        [[-7.56353668e-07,  0.00000000e+00],
         [-3.35491922e-07,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00]]]),
 array([[ 1.21901872e-04,  0.00000000e+00],
        [-1.20810027e-04,  0.00000000e+00],
        [-1.09184559e-06,  0.00000000e+00]]))